# Modeling How to Beat the Streak

In 1941, Joe DiMaggio recorded hits in 56 consecutive games of Major League Baseball. This record has stood for 80 years and is seen as one of the most unlikely to be broken. In 2001, MLB started a fantasy game called Beat the Streak, challenging fans to simply pick, among all player, one who would get a hit on a given day. And then do that for 57 consecutive days to symbolically beat DiMaggio's streak. In 20 years nobody has won but a few have come as close as 51.

In this notebook I aim to take a statistical approach to determining these daily picks in order to give myself the best possible chance I can to win this competition and, with it, the $5.6 million prize.

My first step will be to gather the relevant data and assemble it in a useful way.

In [18]:
import os
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import statsapi
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')


import pybaseball

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, precision_score, plot_roc_curve
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier


import xgboost as xgb

import category_encoders as ce

from dateutil import tz

from time import sleep

import pickle

pybaseball.statcast allows me to gather details from of every MLB game since 2017 down to the individual pitch level. As this alone is a lot of data I'll save it as I gather it so I don't have to rerun this part. 

As I take it in I'll transform it to save on memory since the dataset is quite large and will take a while to run.

In [19]:
def statcast_importer(year = datetime.now().year):
    start = datetime(year, 1, 1)
    if year == datetime.now().year:
        end = datetime.now()-timedelta(weeks=1)
    else:
        end = datetime(year,12, 31)
    df = pybaseball.statcast(str(start.year) + '-' + '{:0>2}'.format(str(start.month)) + '-' + '{:0>2}'.format(str(start.day)), str(end.year) + '-' + '{:0>2}'.format(str(end.month)) + '-' + '{:0>2}'.format(str(end.day)))
    df.to_csv('data/untouched_' + str(year) + '_statcast_pbp.csv')

    num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

    num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median'))
    ])
    preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_columns)])
    clf = Pipeline(steps=[('preprocessor', preprocessor)])

    t = clf.fit_transform(df)

    df[num_columns] = t

#   drop spring training, playoff games

    df = df[df['game_type'] == 'R']
    print('Non-regular season games dropped')


    df.to_csv('data/'+str(year)+'_statcast_pbp.csv')
    return(df)

In [20]:
# establish definitions for the events column for what ends a plate appearance ending event and a hit
# the statcast includes data for every pitch which needs to be pared down to determine whether a player got a hit
pa_ending_events = np.array(['field_out',
                        'strikeout',
                        'single',
                        'walk',
                        'double',
                        'home_run',
                        'force_out',
                        'grounded_into_double_play',
                        'hit_by_pitch',
                        'field_error',
                        'sac_fly',
                        'triple',
                        'sac_bunt',
                        'fielders_choice',
                        'double_play',
                        'fielders_choice_out',
                        'strikeout_double_play',
                        'catcher_interf',
                        'sac_fly_double_play',
                        'triple_play',
                        'sac_bunt_double_play'])


hit_events = np.array(['single',
            'double',
            'home_run',
            'triple'])

american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])


num_columns = ['release_speed', 'batter', 'pitcher', 'zone', 'hit_location', 'balls', 'strikes', 'game_year', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'fielder_2', 'hit_distance_sc', 'release_spin_rate', 'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'woba_value', 'woba_denom', 'babip_value', 'at_bat_number', 'pitch_number', 'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score', 'post_home_score', 'post_bat_score', 'post_fld_score']

column_types = [np.int8,
np.int32,
np.int32,
np.int8,
np.int8,
np.int8,
np.int8,
np.int16,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int32,
np.int16,
np.int16,
np.int32,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int16,
np.int8,
np.int8,
np.int8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8,
np.uint8]

type_dict = dict(zip(num_columns, column_types))

This explicity sets types to the numeric features in order to save on memory.

In [21]:
def set_types(df, type_dict):
    '''Explicitly names datatypes in order to speed up preprocessing of a large dataset'''

    df['game_date'] = pd.to_datetime(df['game_date'], format='%Y-%m-%d')
    print('datetime done')
    df['events'] = df['events'].apply(lambda x: 1 if x in hit_events else (0 if x in pa_ending_events else -1)).astype(np.int8)
    print('events done')
    df['batter_righty'] = df['stand'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('stand done')
    df['pitcher_righty'] = df['p_throws'].apply(lambda x: 1 if x == 'R' else 0).astype(np.int8)
    print('pitcher done')
    df['home'] = df['inning_topbot'].apply(lambda x: 1 if x == 'Bot' else 0).astype(np.int8)
    print('inning done')

    for key, value in type_dict.items():
        df[key] = df[key].astype(value)

    df = df.select_dtypes(exclude=['object'])
    for col in df.columns:
        if col.startswith('Unnamed:'):
            df = df.drop([col], axis=1)
    return (df)

Time zone conversion based on venue name (each is unique in Parks.csv) in order to pass the proper times to the Visual Crossing weather api

In [22]:
def convert_UTC_to_local(row):
    """Takes in a row containing a baseball game's location and UTC start time"""

    #     Reads in latitude, longitude, and altitude for known venues
    venue_coords = pd.read_csv('Parks.csv')

    venue_name = row['venue_name']
    game_datetime = row['game_datetime']

    #     get timezone object for UTC
    from_zone = tz.gettz('UTC')

    try:
        city = venue_coords[venue_coords['NAME'] == row['venue_name']]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == row['venue_name']]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == row['venue_name']]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == row['venue_name']]['Altitude'].iloc[0]
    except NameError:
        #         Need to add new venue to Parks.csv
        raise IndexError(venue, index)
    if city == 'Tokyo':
        to_zone = tz.gettz('Asia/Tokyo')
    elif city == 'London':
        to_zone = tz.gettz('Europe/London')
    elif city in ['San Francisco',
                  'Oakland',
                  'Phoenix',
                  'Seattle',
                  'Los Angeles',
                  'San Diego',
                  'Anaheim']:
        to_zone = tz.gettz('America/Los_Angeles')
    elif city == 'Denver':
        to_zone = tz.gettz('America/Denver')
    elif city in ['Minneapolis',
                  'Milwaukee',
                  'Chicago',
                  'St. Louis',
                  'Arlington',
                  'Kansas City',
                  'Houston',
                  'Monterrey',
                  'Omaha',
                  'Dyersville']:
        to_zone = tz.gettz('America/Chicago')
    elif city in ['Buffalo',
                  'Detroit',
                  'Cincinnati',
                  'Pittsburgh',
                  'Tampa Bay',
                  'Philadelphia',
                  'Atlanta',
                  'New York',
                  'Washington',
                  'Cleveland',
                  'Miami',
                  'Boston',
                  'Baltimore',
                  'Toronto',
                  'Williamsport',
                  'Dunedin',
                  'St. Petersburg',
                  'Fort Bragg']:
        to_zone = tz.gettz('America/New_York')
    elif city == San Juan:
        to_zone = tz.gettz('America/Puerto_Rico')
    else:
        raise NameError(venue_name, city)
    utc = datetime.strptime(game_datetime, '%Y-%m-%dT%H:%M:%SZ')
    #     Convert start time from
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

In [14]:
api_counter = 0

Set what portion of the historical statcast data we want to study. We'll start the collection in the 2015 season since that's the first one with launch angle and speed information.

In [15]:
for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021]:
    statcast_importer(year)

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 211/211 [00:21<00:00,  9.63it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 214/214 [00:43<00:00,  4.96it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


...

Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|██████████| 97/97 [00:17<00:00,  5.57it/s]


Non-regular season games dropped
This is a large query, it may take a moment to complete
Skipping offseason dates


100%|██████████| 177/177 [00:34<00:00,  5.20it/s]


Non-regular season games dropped


In [75]:
df = set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)

df = df[df['game_date']<= datetime(2021,6,30)]

df = pd.concat([set_types(pd.read_csv('data/2015_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2016_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2017_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2018_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
                set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
                ], ignore_index=True)

datetime done
events done
stand done
pitcher done
inning done


In [76]:
df_filtered = df[df['events'].isin([1, 0])]


game_pk_df = pd.DataFrame(columns=statsapi.schedule(game_id=566083)[0].keys())

venue_dict = {}

#             grabs metadata for respective games such as projected starting pitchers and venue names
for game in df_filtered['game_pk'].unique():
    print(game)
    game_data = statsapi.schedule(game_id=int(game))[-1]
    game_pk_df = game_pk_df.append(game_data, ignore_index=True)
    print(game)

633506
633506
633491
633491
633485
633485
633484
633484
633479
633479
633474
633474
633467
633467
633461
633461
633445
633445
633437
633437
633435
633435
633432
633432
633427
633427
633426
633426
633419
633419
633587
633587
633508
633508
633497
633497
633496
633496
633495
633495
633488
633488
633478
633478
633471
633471
633455
633455
633454
633454
633449
633449
633446
633446
633444
633444
633443
633443
634620
634620
633867
633867
633601
633601
633490
633490
633483
633483
633475
633475
633466
633466
633465
633465
633458
633458
633450
633450
633544
633544
633527
633527
633516
633516
633515
633515
633504
633504
633499
633499
633494
633494
633493
633493
633489
633489
633486
633486
633482
633482
633477
633477
633473
633473
633472
633472
633459
633459
633589
633589
633562
633562
633552
633552
633539
633539
633534
633534
633531
633531
633530
633530
633525
633525
633520
633520
633518
633518
633512
633512
633510
633510
633509
633509
633501
633501
633500
633500
634515
634515
633620
633620
633608

634052
634051
634051
634050
634050
634043
634043
634101
634101
634099
634099
634091
634091
634082
634082
634077
634077
634076
634076
634070
634070
634063
634063
634057
634057
634188
634188
634134
634134
634130
634130
634124
634124
634122
634122
634100
634100
634098
634098
634094
634094
634090
634090
634087
634087
634086
634086
634083
634083
634071
634071
634069
634069
634068
634068
634161
634161
634148
634148
634143
634143
634139
634139
634127
634127
634111
634111
634106
634106
634105
634105
634104
634104
634097
634097
634096
634096
634095
634095
634085
634085
634084
634084
634080
634080
634656
634656
634206
634206
634166
634166
634162
634162
634146
634146
634145
634145
634138
634138
634132
634132
634131
634131
634123
634123
634121
634121
634119
634119
634118
634118
634110
634110
634108
634108
634093
634093
634191
634191
634158
634158
634150
634150
634141
634141
634136
634136
634135
634135
634126
634126
634125
634125
634117
634117
634114
634114
634113
634113
634103
634103
634238
634238

634596
634580
634580
634577
634577
634575
634575
634573
634573
634653
634653
634631
634631
634627
634627
634605
634605
634602
634602
634578
634578
634576
634576
634655
634655
634652
634652
634645
634645
634642
634642
634640
634640
634638
634638
634636
634636
634634
634634
634630
634630
634625
634625
634622
634622
634618
634618
634615
634615


In [77]:
#             grabs coordinate date for venues from Parks.csv which needs to be manually updated every time
#             there's a new venue or name change
venue_coords = pd.read_csv('Parks.csv')

for venue in game_pk_df['venue_name'].unique():
    print(venue)
    try:
        city = venue_coords[venue_coords['NAME'] == venue]['CITY'].iloc[0]
        lat = venue_coords[venue_coords['NAME'] == venue]['Latitude'].iloc[0]
        lon = venue_coords[venue_coords['NAME'] == venue]['Longitude'].iloc[0]
        alt = venue_coords[venue_coords['NAME'] == venue]['Altitude'].iloc[0]
        venue_dict[venue] = np.array((city, lat, lon, alt))
    except IndexError:
        raise IndexError(venue)

#             converts game start times from UTC to local times, based on hand written mapping
#             in convert_UTC_to_local for the purposes of collating start times to cut down on
#             Visual Crossing api calls as they cost money after the first 1000 per day
#

game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))

datetime_coordinate_matching = {}
for index, row in game_pk_df.iterrows():
    city, lat, lon, alt = venue_dict[row['venue_name']]
    dc_datetime = row['local_datetime']
    if dc_datetime in datetime_coordinate_matching.keys():
        datetime_coordinate_matching[dc_datetime].append(','.join([str(lat), str(lon)]))
    else:
        datetime_coordinate_matching[dc_datetime] = [','.join([str(lat), str(lon)])]

game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join(venue_dict[x][1:3]))
game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_dict[x][3])

#             calls Visual Crossing for historical weather data for each game

Progressive Field
Coors Field
Minute Maid Park
Yankee Stadium
Guaranteed Rate Field
American Family Field
Fenway Park
Oakland Coliseum
Sahlen Field
Truist Park
Great American Ball Park
Nationals Park
Busch Stadium
Citizens Bank Park
Dodger Stadium
Petco Park
Oracle Park
Globe Life Field
Citi Field
Comerica Park
loanDepot park
Target Field
Tropicana Field
T-Mobile Park
Chase Field
Angel Stadium
PNC Park
Oriole Park at Camden Yards
Wrigley Field
Kauffman Stadium
TD Ballpark


In [78]:
datetime_coordinate_matching

{'2021-06-29T23:33:00': ['41.496005,-81.685326'],
 '2021-06-30T13:10:00': ['39.756175,-104.99413', '43.028232,-87.970966'],
 '2021-06-30T19:10:00': ['29.757041,-95.355429',
  '41.829892,-87.633703',
  '42.346561,-71.097337',
  '39.097213,-84.506483'],
 '2021-06-30T19:05:00': ['40.829586,-73.926413', '39.906109,-75.166485'],
 '2021-06-30T18:40:00': ['37.751619,-122.200451'],
 '2021-06-30T19:07:00': ['42.881306,-78.874278'],
 '2021-06-30T19:20:00': ['33.89127,-84.4681'],
 '2021-06-30T16:10:00': ['41.496005,-81.685326'],
 '2021-06-30T16:05:00': ['38.872987,-77.007435'],
 '2021-06-30T12:15:00': ['38.622622,-90.192841'],
 '2021-06-29T19:20:00': ['33.89127,-84.4681'],
 '2021-06-29T19:15:00': ['38.622622,-90.192841'],
 '2021-06-29T19:07:00': ['42.881306,-78.874278'],
 '2021-06-29T19:10:00': ['42.346561,-71.097337',
  '41.829892,-87.633703',
  '34.073878,-118.239951',
  '39.097213,-84.506483',
  '29.757041,-95.355429',
  '43.028232,-87.970966'],
 '2021-06-29T19:05:00': ['39.906109,-75.166485',

In [79]:
from api_keys import visual_crossing
import requests

weather_df = pd.DataFrame()
for key, value in datetime_coordinate_matching.items():
    print(key)
    url_locations = '|'.join(value)
    URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history' \
          f'?&aggregateHours=1&startDateTime={key}&endDateTime={key}&unitGroup=us&contentType=json&loc' \
          f'ation={url_locations}&key={visual_crossing} '
    if api_counter == 1000:
        print(
            'Prepreprocessing will continue at ' + (datetime.now() + timedelta(seconds=86400)).strftime('%H:%M:%S'))
        sleep(86400)
        api_counter = 0
        api_counter += 1
        response = requests.get(URL)
    else:
        api_counter += 1
        response = requests.get(URL)
    try:
        data = response.json()
        locations = list(data['locations'].keys())
        for each in locations:
            values = data['locations'][each]['values'][0]

            # creates a df containing coordinate data and returned weather data to later be added with other
            # collected data

            weather_df = weather_df.append({**{'coordinates': each}, **values}, ignore_index=True)
    except:
        raise IndexError(key, value)
#             cuts off the trailing stamp in datetimeStr signifying the time zone (e.g.-05:00)
weather_df['datetimeStr'] = weather_df['datetimeStr'].apply(lambda x: x[:-6])
weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})

weather_df.to_csv('weather_df')

2021-06-29T23:33:00
2021-06-30T13:10:00
2021-06-30T19:10:00
2021-06-30T19:05:00
2021-06-30T18:40:00
2021-06-30T19:07:00
2021-06-30T19:20:00
2021-06-30T16:10:00
2021-06-30T16:05:00
2021-06-30T12:15:00
2021-06-29T19:20:00
2021-06-29T19:15:00
2021-06-29T19:07:00
2021-06-29T19:10:00
2021-06-29T19:05:00
2021-06-29T18:40:00
2021-06-28T19:05:00
2021-06-28T18:40:00
2021-06-28T19:15:00
2021-06-28T19:10:00
2021-06-28T15:10:00
2021-06-27T13:10:00
2021-06-27T13:05:00
2021-06-27T13:35:00
2021-06-27T13:07:00
2021-06-27T16:15:00
2021-06-27T13:15:00
2021-06-27T16:08:00
2021-06-26T13:15:00
2021-06-26T16:15:00
2021-06-26T16:10:00
2021-06-26T15:05:00
2021-06-26T19:15:00
2021-06-26T13:10:00
2021-06-26T15:07:00
2021-06-26T15:10:00
2021-06-26T19:10:00
2021-06-26T18:10:00
2021-06-26T19:05:00
2021-06-24T23:33:00
2021-06-25T19:10:00
2021-06-25T15:10:00
2021-06-25T19:05:00
2021-06-25T16:10:00
2021-06-25T19:07:00
2021-06-25T18:45:00
2021-06-25T19:15:00
2021-06-24T19:10:00
2021-06-24T19:07:00
2021-06-24T13:05:00


2021-05-04T13:20:00
2021-05-03T21:33:00
2021-05-04T15:40:00
2021-05-03T18:38:00
2021-05-03T18:40:00
2021-05-03T18:45:00
2021-05-03T19:10:00
2021-05-03T19:05:00
2021-05-02T13:10:00
2021-05-02T13:05:00
2021-05-02T13:07:00
2021-05-02T13:35:00
2021-05-02T19:08:00
2021-05-01T18:10:00
2021-05-01T13:10:00
2021-05-01T15:05:00
2021-05-01T13:07:00
2021-05-01T17:40:00
2021-05-01T19:37:00
2021-05-01T16:10:00
2021-05-01T13:05:00
2021-05-01T17:10:00
2021-05-01T18:35:00
2021-05-01T18:05:00
2021-04-30T19:10:00
2021-04-30T18:40:00
2021-04-30T19:05:00
2021-04-30T18:35:00
2021-04-30T19:37:00
2021-04-29T18:40:00
2021-04-28T22:33:00
2021-04-29T13:10:00
2021-04-29T12:15:00
2021-04-29T19:20:00
2021-04-29T16:10:00
2021-04-29T13:05:00
2021-04-29T19:05:00
2021-04-28T19:20:00
2021-04-28T13:10:00
2021-04-28T19:05:00
2021-04-28T19:10:00
2021-04-28T19:07:00
2021-04-28T18:40:00
2021-04-28T12:40:00
2021-04-28T18:35:00
2021-04-28T18:45:00
2021-04-27T19:10:00
2021-04-27T19:05:00
2021-04-27T18:40:00
2021-04-27T18:45:00


In [80]:
games_and_weather = pd.merge(
    game_pk_df,
    weather_df,
    how="left",
    on=None,
    left_on=['local_datetime', 'coordinates'],
    right_on=['local_datetime', 'coordinates'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gpk", "_acw"),
    copy=True,
    indicator=False,
    validate=None,
)

df_detailed = pd.merge(
    games_and_weather,
    df_filtered,
    how="right",
    on=None,
    left_on='game_id',
    right_on='game_pk',
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_gaw", "_dff"),
    copy=True,
    indicator=False,
    validate=None,
)

In [81]:
#             additional useful data added to df
df_detailed['covid_doubleheader'] = df_detailed.apply(
    lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] == 'Y' else 0, axis=1)
df_detailed['designated_hitter'] = df_detailed.apply(
    lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis=1)

#         read in entire relevant history (2 years prior to game in data) in order to assemble
#         cumulative stats for players
history = pd.concat([
    set_types(pd.read_csv('data/2017_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2018_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
], ignore_index=True)

#         pitcher_df combines away and home pitchers into one column
#         each row is a projected starting pitcher and the game they're
#         starting as well as the date
pitcher_df = pd.concat([df_detailed[['game_id', 'game_date_gaw', 'away_probable_pitcher']].rename(
    columns={'away_probable_pitcher': 'probable_pitcher'}),
                        df_detailed[['game_id', 'game_date_gaw', 'home_probable_pitcher']].rename(
                            columns={'home_probable_pitcher': 'probable_pitcher'})])
pitcher_df = pitcher_df.drop_duplicates()


datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done


In [82]:
def derived_cumulative_stats(player, stats_type, history, start_date=None,
                             end_date=datetime.strptime('2021-06-30', '%Y-%m-%d')) -> pd.DataFrame:

    """Takes in a player and returns a dataframe containing the player's stats derived from their play over the prior two years"""

    print(end_date)
    print(player)

    #    statistics meant to describe last 2 years

    if start_date is None:
        start_date = end_date - timedelta(weeks=102)

    #    if player's name is passed in, rather than their mlbam ID, lookup ID

    if type(player) == str:
        if player[-3:] == 'Jr.' or player[-3:] == 'III':
            player = player[:-4]
        player = player.split(' ')
        player.reverse()
        try:
            player_info = pybaseball.playerid_lookup(player[0], player[-1])
            if len(player_info) > 1:
                player_info = player_info.iloc[0]
                player = player_info['key_mlbam']
            elif player_info.empty:
                player_info = pybaseball.playerid_lookup(player[0], player[-1], fuzzy=True).iloc[0]
                player = player_info['key_mlbam']
            else:
                try:
                    player = player_info.iloc[0]['key_mlbam']
                except:
                    raise NameError(player_info, player)
        except:
            raise NameError(player)

    history = history[(history['game_date'] >= start_date) & (history['game_date'] <= end_date)]

    history = history[history['events'].isin([1, 0])]

    #     if player is a pitcher, send down derive_pitcher path
    if stats_type == 'pitcher':
        return derive_pitcher(player, history)
    elif stats_type == 'batter':
        return derive_batter(player, history)

def derive_batter(player, history):
    try:
        history = history[history['batter'] == player]
    except:
        raise NameError(player)
    #     H/PA per pitch type
    #     K/PA per pitch type
    #     H/PA vs righties, lefties
    #     PA/G
    #     avg_launch_angle
    #     avg_launch_speed
    #     xBA based on estimated_ba_using_speedangle

    games = history['game_pk'].unique()
    pas_list = []

    batter = {}

    #         find average number of plate appearances per game
    for game in games:
        pas = len(history[history['game_pk'] == game])
        pas_list.append(pas)
    batter['games_played_last_2_years_batter'] = len(games)
    batter['PA/G_batter'] = np.array(pas_list).mean()

    #         righty/lefty split
    pitcher_right = history[history['pitcher_righty'] == 1]
    pas_r = len(pitcher_right)
    hits_r = len(pitcher_right[pitcher_right['events'] == 1])

    batter['H/PA_against_R'] = hits_r / pas_r if pas_r > 0 else 0
    pitcher_left = history[history['pitcher_righty'] == 0]
    pas_l = len(pitcher_left)
    hits_l = len(pitcher_left[pitcher_left['events'] == 1])

    batter['H/PA_against_L'] = hits_l / pas_l if pas_l > 0 else 0
    if pas_r > 0 or pas_l > 0:
        batter['H/PA_batter'] = (hits_r + hits_l) / (pas_r + pas_l)
    else:
        batter['H/PA_batter'] = 0
    batter['avg_launch_angle'] = history['launch_angle'].mean()
    batter['avg_launch_speed'] = history['launch_speed'].mean()
    batter['xBA'] = history['estimated_ba_using_speedangle'].mean()

    return (batter)

def derive_pitcher(player, history):
    try:
        history = history[history['pitcher'] == player]
    except:
        raise NameError(player)
    print('player filtered')
    #   PITCH METRICS
    pitches = {}
    while True:
        try:
            pitches['pitch_hand'] = statsapi.player_stat_data(player)['pitch_hand']
            break
        except:
            print('pitch_hand_error')
            sleep(10)

    #     filter history down to plate appearance ending plays

    games = history['game_pk'].unique()
    plate_appearances_list = []
    num_hits_list = []
    inning_list = []
    for game in games:
        game = history[history['game_pk'] == game]
        inning = game['inning'].max()
        inning_list.append(inning)
        plate_appearances = len(game)
        plate_appearances_list.append(plate_appearances)
        #             for each inning, collect number of hit events to determine average inning
        for i in range(1, inning + 1):
            num_hits = len(game[(game['inning'] == i) & (game['events'] == 1)])
            num_hits_list.append(num_hits)
    pitches['games_played_last_2_years_pitcher'] = len(games)
    pitches['avg_innings_per_appearance'] = np.array(inning_list).mean()
    pitches['avg_PAs_per_appearance_pitcher'] = np.array(plate_appearances_list).mean()
    pitches['avg_hits_per_inning'] = np.array(num_hits_list).mean()

    #         L/R splits
    left_hitter = history[history['batter_righty'] == 0]
    right_hitter = history[history['batter_righty'] == 1]
    l_pas = len(left_hitter)
    r_pas = len(right_hitter)
    l_hits = len(left_hitter[(history['events'] == 1)])
    r_hits = len(right_hitter[(history['events'] == 1)])

    if l_pas > 0 and r_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = (l_hits) / (l_pas)
        pitches['against_righties_H/PA'] = (r_hits) / (r_pas)
    elif l_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = (l_hits) / (l_pas)
        pitches['against_righties_H/PA'] = 0
    elif r_pas > 0:
        pitches['H/PA_pitcher'] = (l_hits + r_hits) / (l_pas + r_pas)
        pitches['against_lefties_H/PA'] = 0
        pitches['against_righties_H/PA'] = (r_hits) / (r_pas)
    else:
        pitches['H/PA_pitcher'] = 0
        pitches['against_lefties_H/PA'] = 0
        pitches['against_righties_H/PA'] = 0
    return (pitches)

In [83]:
#         if probable pitcher is unknown, drop row
pitcher_df['probable_pitcher'] = pitcher_df['probable_pitcher'].replace('', np.nan)
pitcher_df = pitcher_df.dropna(subset=['probable_pitcher'])
pitcher_df = pitcher_df.reset_index(drop=True)

#         runs pitcher_df through derived_cumulative_stats function to get their recent history
pitcher_stats = pitcher_df.apply(lambda row: pd.Series(
    derived_cumulative_stats(row['probable_pitcher'], 'pitcher', history,
                                end_date=datetime.strptime(row['game_date_gaw'], '%Y-%m-%d') - timedelta(days=1))),
                                 axis=1)

pitcher_df = pd.merge(
    pitcher_stats,
    pitcher_df,
    how="left",
    on=None,
    left_on=None,
    right_on=None,
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("_s", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)


2021-06-29 00:00:00
Wily Peralta
player filtered
2021-06-29 00:00:00
Chad Kuhl
player filtered
2021-06-29 00:00:00
Matt Harvey
player filtered
2021-06-29 00:00:00
Shohei Ohtani
player filtered
2021-06-29 00:00:00
Bailey Ober
player filtered
2021-06-29 00:00:00
Jake Arrieta
player filtered
2021-06-29 00:00:00
Mike Minor
player filtered
2021-06-29 00:00:00
Kolby Allard
player filtered
2021-06-29 00:00:00
Justus Sheffield
player filtered
2021-06-29 00:00:00
David Peterson
player filtered
2021-06-29 00:00:00
Joe Musgrove
player filtered
2021-06-29 00:00:00
Jose Urena
player filtered
2021-06-29 00:00:00
Drew Rasmussen
player filtered
2021-06-29 00:00:00
Riley Smith
player filtered
2021-06-29 00:00:00
Jordan Holloway
player filtered
2021-06-28 00:00:00
Tylor Megill
player filtered
2021-06-28 00:00:00
Caleb Smith
player filtered
2021-06-28 00:00:00
Chris Flexen
player filtered
2021-06-28 00:00:00
Brad Keller
player filtered
2021-06-28 00:00:00
Trevor Rogers
player filtered
2021-06-28 00:00:00

2021-06-18 00:00:00
Robert Gsellman
player filtered
2021-06-18 00:00:00
Adrian Houser
player filtered
2021-06-18 00:00:00
Lance Lynn
player filtered
2021-06-18 00:00:00
Alek Manoah
player filtered
2021-06-17 00:00:00
Carlos Rodon
player filtered
2021-06-17 00:00:00
Nick Pivetta
player filtered
2021-06-17 00:00:00
Jose Urena
player filtered
2021-06-17 00:00:00
Cody Poteet
player filtered
2021-06-17 00:00:00
Joey Lucchesi
player filtered
2021-06-17 00:00:00
Corbin Burnes
player filtered
2021-06-17 00:00:00
Michael Wacha
player filtered
2021-06-17 00:00:00
James Kaprielian
player filtered
2021-06-17 00:00:00
Carlos Martinez
player filtered
2021-06-17 00:00:00
Trevor Bauer
player filtered
2021-06-17 00:00:00
Vince Velasquez
player filtered
2021-06-17 00:00:00
Tony Santillan
player filtered
2021-06-17 00:00:00
Robbie Ray
player filtered
2021-06-17 00:00:00
J.C. Mejia
No identically matched names found! Returning the 5 most similar names.
player filtered
2021-06-17 00:00:00
Jose Berrios
play

player filtered
2021-06-07 00:00:00
Zach Davies
player filtered
2021-06-07 00:00:00
David Peterson
player filtered
2021-06-07 00:00:00
Walker Buehler
player filtered
2021-06-07 00:00:00
Jon Lester
player filtered
2021-06-07 00:00:00
Jon Duplantier
player filtered
2021-06-07 00:00:00
Alex Wood
player filtered
2021-06-06 00:00:00
Zach Thompson
player filtered
2021-06-06 00:00:00
Jackson Kowar
player filtered
2021-06-06 00:00:00
Adbert Alzolay
player filtered
2021-06-05 00:00:00
Bailey Ober
player filtered
2021-06-05 00:00:00
Kyle Hendricks
player filtered
2021-06-05 00:00:00
James Kaprielian
player filtered
2021-06-05 00:00:00
Logan Gilbert
player filtered
2021-06-05 00:00:00
Cal Quantrill
player filtered
2021-06-05 00:00:00
Garrett Richards
player filtered
2021-06-05 00:00:00
Trevor Bauer
player filtered
2021-06-05 00:00:00
Wade Miley
player filtered
2021-06-05 00:00:00
Marcus Stroman
player filtered
2021-06-05 00:00:00
Michael Wacha
player filtered
2021-06-05 00:00:00
Caleb Smith
playe

player filtered
2021-05-26 00:00:00
Bruce Zimmermann
player filtered
2021-05-26 00:00:00
German Marquez
player filtered
2021-05-26 00:00:00
Brady Singer
player filtered
2021-05-26 00:00:00
Carlos Martinez
player filtered
2021-05-26 00:00:00
Antonio Senzatela
player filtered
2021-05-26 00:00:00
Robbie Ray
player filtered
2021-05-26 00:00:00
Kyle Hendricks
player filtered
2021-05-25 00:00:00
Robert Dugger
player filtered
2021-05-25 00:00:00
Mike Minor
player filtered
2021-05-25 00:00:00
Trevor Bauer
player filtered
2021-05-25 00:00:00
Drew Smyly
player filtered
2021-05-25 00:00:00
Triston McKenzie
player filtered
2021-05-25 00:00:00
John Gant
player filtered
2021-05-25 00:00:00
Trevor Williams
player filtered
2021-05-25 00:00:00
Dane Dunning
player filtered
2021-05-25 00:00:00
Johnny Cueto
player filtered
2021-05-25 00:00:00
Jorge Lopez
player filtered
2021-05-25 00:00:00
Chris Paddack
player filtered
2021-05-25 00:00:00
Aaron Nola
player filtered
2021-05-26 00:00:00
Jeff Hoffman
player 

player filtered
2021-05-14 00:00:00
Trevor Williams
player filtered
2021-05-14 00:00:00
Ian Anderson
player filtered
2021-05-14 00:00:00
Drew Smith
player filtered
2021-05-14 00:00:00
Domingo German
player filtered
2021-05-14 00:00:00
Joe Ross
player filtered
2021-05-14 00:00:00
Dylan Bundy
player filtered
2021-05-14 00:00:00
Johnny Cueto
player filtered
2021-05-14 00:00:00
Triston McKenzie
player filtered
2021-05-14 00:00:00
Cole Irvin
player filtered
2021-05-13 00:00:00
Brad Keller
player filtered
2021-05-13 00:00:00
Jake Arrieta
player filtered
2021-05-13 00:00:00
Jakob Junis
player filtered
2021-05-13 00:00:00
Max Scherzer
player filtered
2021-05-13 00:00:00
Corey Kluber
player filtered
2021-05-13 00:00:00
Sandy Alcantara
player filtered
2021-05-13 00:00:00
Griffin Canning
player filtered
2021-05-13 00:00:00
Johan Oviedo
player filtered
2021-05-13 00:00:00
Frankie Montas
player filtered
2021-05-13 00:00:00
Wes Benjamin
player filtered
2021-05-13 00:00:00
Wade Miley
player filtered


...

player filtered
2021-05-22 00:00:00
Patrick Corbin
player filtered
2021-05-22 00:00:00
Hyun Jin Ryu
No identically matched names found! Returning the 5 most similar names.
player filtered
2021-05-22 00:00:00
Adam Wainwright
player filtered
2021-05-21 00:00:00
Robbie Ray
player filtered
2021-05-21 00:00:00
Jordan Lyles
player filtered
2021-05-21 00:00:00
Jon Lester
player filtered
2021-05-21 00:00:00
Bryse Wilson
player filtered
2021-05-21 00:00:00
Shane Bieber
player filtered
2021-05-21 00:00:00
Gerrit Cole
player filtered
2021-05-21 00:00:00
Miles Mikolas
player filtered
2021-05-21 00:00:00
Sonny Gray
player filtered
2021-05-21 00:00:00
Scott Kazmir
player filtered
2021-05-21 00:00:00
Antonio Senzatela
player filtered
2021-05-21 00:00:00
Brady Singer
player filtered
2021-05-21 00:00:00
Patrick Sandoval
player filtered
2021-05-21 00:00:00
Pablo Lopez
player filtered
2021-05-21 00:00:00
Ryan Weathers
player filtered
2021-05-21 00:00:00
Spencer Howard
player filtered
2021-05-20 00:00:00


2021-05-10 00:00:00
Madison Bumgarner
player filtered
2021-05-10 00:00:00
Matthew Boyd
No identically matched names found! Returning the 5 most similar names.
player filtered
2021-05-10 00:00:00
JT Brubaker
player filtered
2021-05-10 00:00:00
Marcus Stroman
player filtered
2021-05-10 00:00:00
Luis Patino
player filtered
2021-05-10 00:00:00
Logan Webb
player filtered
2021-05-10 00:00:00
Bryse Wilson
player filtered
2021-05-10 00:00:00
Freddy Peralta
player filtered
2021-05-10 00:00:00
Lance McCullers Jr.
player filtered
2021-05-10 00:00:00
Antonio Senzatela
player filtered
2021-05-10 00:00:00
Dylan Cease
player filtered
2021-05-10 00:00:00
Erick Fedde
player filtered
2021-05-10 00:00:00
Walker Buehler
player filtered
2021-05-09 00:00:00
Luis Garcia
player filtered
2021-05-09 00:00:00
Jorge Lopez
player filtered
2021-05-09 00:00:00
Luke Weaver
player filtered
2021-05-09 00:00:00
Alex Wood
player filtered
2021-05-09 00:00:00
Mitch Keller
player filtered
2021-05-08 00:00:00
Kyle Hendricks


player filtered
2021-04-28 00:00:00
Eric Lauer
player filtered
2021-04-28 00:00:00
Carlos Rodon
player filtered
2021-04-28 00:00:00
Jorge Lopez
player filtered
2021-04-28 00:00:00
Shane McClanahan
player filtered
2021-04-28 00:00:00
Kyle Gibson
player filtered
2021-04-27 00:00:00
Huascar Ynoa
player filtered
2021-04-27 00:00:00
Logan Allen
player filtered
2021-04-27 00:00:00
Clayton Kershaw
player filtered
2021-04-27 00:00:00
Dean Kremer
player filtered
2021-04-27 00:00:00
Zack Greinke
player filtered
2021-04-27 00:00:00
Dane Dunning
player filtered
2021-04-27 00:00:00
Steven Matz
player filtered
2021-04-27 00:00:00
Riley Smith
player filtered
2021-04-27 00:00:00
Zack Godley
player filtered
2021-04-27 00:00:00
Mitch Keller
player filtered
2021-04-27 00:00:00
Johan Oviedo
player filtered
2021-04-27 00:00:00
Jacob deGrom
player filtered
2021-04-27 00:00:00
Alex Wood
player filtered
2021-04-27 00:00:00
Tyler Glasnow
player filtered
2021-04-26 00:00:00
David Peterson
player filtered
2021-0

2021-04-16 00:00:00
Matt Moore
player filtered
2021-04-16 00:00:00
Trevor Williams
player filtered
2021-04-16 00:00:00
Sonny Gray
player filtered
2021-04-16 00:00:00
Sandy Alcantara
player filtered
2021-04-16 00:00:00
Yu Darvish
player filtered
2021-04-16 00:00:00
Chris Flexen
player filtered
2021-04-15 00:00:00
Zach Davies
player filtered
2021-04-15 00:00:00
Ryan Weathers
player filtered
2021-04-15 00:00:00
Andrew Heaney
player filtered
2021-04-15 00:00:00
Frankie Montas
player filtered
2021-04-15 00:00:00
Daniel Castano
player filtered
2021-04-15 00:00:00
Yusei Kikuchi
player filtered
2021-04-15 00:00:00
Adrian Houser
player filtered
2021-04-15 00:00:00
Jeff Hoffman
player filtered
2021-04-15 00:00:00
Zach Eflin
player filtered
2021-04-15 00:00:00
Max Scherzer
player filtered
2021-04-15 00:00:00
Nick Nelson
player filtered
2021-04-15 00:00:00
Mike Foltynewicz
player filtered
2021-04-14 00:00:00
Sean Manaea
player filtered
2021-04-14 00:00:00
Mitch Keller
player filtered
2021-04-14 00

player filtered
2021-04-04 00:00:00
Justus Sheffield
player filtered
2021-04-03 00:00:00
Sean Manaea
player filtered
2021-04-03 00:00:00
Shohei Ohtani
player filtered
2021-04-03 00:00:00
Zach Eflin
player filtered
2021-04-03 00:00:00
Zach Davies
player filtered
2021-04-03 00:00:00
Tarik Skubal
player filtered
2021-04-03 00:00:00
Garrett Richards
player filtered
2021-04-03 00:00:00
Domingo German
player filtered
2021-04-03 00:00:00
Adrian Houser
player filtered
2021-04-03 00:00:00
Brady Singer
player filtered
2021-04-03 00:00:00
Chris Paddack
player filtered
2021-04-03 00:00:00
Jeff Hoffman
player filtered
2021-04-03 00:00:00
Austin Gomber
player filtered
2021-04-02 00:00:00
Tyler Mahle
player filtered
2021-04-02 00:00:00
Alex Cobb
player filtered
2021-04-02 00:00:00
Corey Kluber
player filtered
2021-04-02 00:00:00
Jake Arrieta
player filtered
2021-04-02 00:00:00
Chris Flexen
player filtered
2021-04-02 00:00:00
Jon Gray
player filtered
2021-04-02 00:00:00
Cole Irvin
player filtered
2021

In [84]:
#         re-matches home batters with away pitchers and vice versa

home_batters = df_detailed[df_detailed['home'] == 1]
away_batters = df_detailed[df_detailed['home'] == 0]

df_wpitching_p1 = pd.merge(
    pitcher_df,
    home_batters,
    how="right",
    on=None,
    left_on=['game_id', 'probable_pitcher'],
    right_on=['game_id', 'away_probable_pitcher'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_p", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)

df_wpitching_p2 = pd.merge(
    pitcher_df,
    away_batters,
    how="right",
    on=None,
    left_on=['game_id', 'probable_pitcher'],
    right_on=['game_id', 'home_probable_pitcher'],
    left_index=False,
    right_index=False,
    sort=False,
    suffixes=("_p", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)

df_wpitching = pd.concat([df_wpitching_p1, df_wpitching_p2], ignore_index=True)


In [85]:
df_wpitching = df_wpitching.sort_values(
    ['game_date_gaw_df', 'game_pk', 'home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number',
     'pitch_number']).reset_index(drop=True)

df_wpitching = df_wpitching.rename(columns={'events': 'got_a_hit'})

#         drops repeat batters in the same game, while preserving the 1 value for got_a_hit, if present
df_wpitching = df_wpitching.sort_values('got_a_hit').drop_duplicates(subset=['game_id', 'batter'],
                                                                     keep='last').sort_values(
    ['game_date_gaw_df', 'game_pk', 'home_name', 'inning', 'home', 'outs_when_up', 'at_bat_number',
     'pitch_number']).reset_index(drop=True)

#     combines batting stats with pitching/statcast/statsapi/weather data

batting_stats = df_wpitching.apply(lambda row: pd.Series(
    derived_cumulative_stats(row['batter'], 'batter', history,
                                end_date=datetime.strptime(row['game_date_gaw_df'], '%Y-%m-%d') - timedelta(days=1))),
                                   axis=1)

2021-03-31 00:00:00
608369
2021-03-31 00:00:00
621035
2021-03-31 00:00:00
477132
2021-03-31 00:00:00
457759
2021-03-31 00:00:00
641355
2021-03-31 00:00:00
572008
2021-03-31 00:00:00
641658
2021-03-31 00:00:00
608566
2021-03-31 00:00:00
658069
2021-03-31 00:00:00
605131
2021-03-31 00:00:00
453568
2021-03-31 00:00:00
666158
2021-03-31 00:00:00
592346
2021-03-31 00:00:00
596115
2021-03-31 00:00:00
605141
2021-03-31 00:00:00
571970
2021-03-31 00:00:00
543068
2021-03-31 00:00:00
553869
2021-03-31 00:00:00
602074
2021-03-31 00:00:00
606132
2021-03-31 00:00:00
656716
2021-03-31 00:00:00
641857
2021-03-31 00:00:00
621458
2021-03-31 00:00:00
669257
2021-03-31 00:00:00
607461
2021-03-31 00:00:00
572233
2021-03-31 00:00:00
500871
2021-03-31 00:00:00
592518
2021-03-31 00:00:00
518516
2021-03-31 00:00:00
668942
2021-03-31 00:00:00
595777
2021-03-31 00:00:00
605170
2021-03-31 00:00:00
506433
2021-03-31 00:00:00
606466
2021-03-31 00:00:00
444482
2021-03-31 00:00:00
543333
2021-03-31 00:00:00
571976
2

2021-04-01 00:00:00
668942
2021-04-01 00:00:00
642165
2021-04-01 00:00:00
641584
2021-04-01 00:00:00
620443
2021-04-01 00:00:00
457763
2021-04-01 00:00:00
571745
2021-04-01 00:00:00
664034
2021-04-01 00:00:00
657108
2021-04-01 00:00:00
456781
2021-04-01 00:00:00
641487
2021-04-01 00:00:00
643289
2021-04-01 00:00:00
446334
2021-04-01 00:00:00
573131
2021-04-01 00:00:00
527038
2021-04-01 00:00:00
600303
2021-04-01 00:00:00
474832
2021-04-01 00:00:00
543063
2021-04-01 00:00:00
543105
2021-04-01 00:00:00
572122
2021-04-01 00:00:00
666211
2021-04-01 00:00:00
596103
2021-04-01 00:00:00
573262
2021-04-01 00:00:00
664238
2021-04-01 00:00:00
668227
2021-04-01 00:00:00
641154
2021-04-01 00:00:00
500743
2021-04-01 00:00:00
516782
2021-04-01 00:00:00
642336
2021-04-01 00:00:00
642232
2021-04-01 00:00:00
665862
2021-04-01 00:00:00
640457
2021-04-01 00:00:00
621446
2021-04-01 00:00:00
622534
2021-04-01 00:00:00
572816
2021-04-01 00:00:00
660294
2021-04-01 00:00:00
542583
2021-04-01 00:00:00
643265
2

2021-04-02 00:00:00
543068
2021-04-02 00:00:00
641857
2021-04-02 00:00:00
621458
2021-04-02 00:00:00
605131
2021-04-02 00:00:00
572041
2021-04-02 00:00:00
666158
2021-04-02 00:00:00
658069
2021-04-02 00:00:00
656541
2021-04-02 00:00:00
656716
2021-04-02 00:00:00
621035
2021-04-02 00:00:00
641658
2021-04-02 00:00:00
596115
2021-04-02 00:00:00
571970
2021-04-02 00:00:00
607461
2021-04-02 00:00:00
553869
2021-04-02 00:00:00
572008
2021-04-02 00:00:00
572122
2021-04-02 00:00:00
592200
2021-04-02 00:00:00
664034
2021-04-02 00:00:00
456781
2021-04-02 00:00:00
543105
2021-04-02 00:00:00
596103
2021-04-02 00:00:00
657108
2021-04-02 00:00:00
666211
2021-04-02 00:00:00
664238
2021-04-02 00:00:00
641584
2021-04-02 00:00:00
620443
2021-04-02 00:00:00
641487
2021-04-02 00:00:00
543063
2021-04-02 00:00:00
600303
2021-04-02 00:00:00
474832
2021-04-02 00:00:00
446334
2021-04-02 00:00:00
571745
2021-04-02 00:00:00
527038
2021-04-02 00:00:00
573262
2021-04-02 00:00:00
573131
2021-04-02 00:00:00
663647
2

2021-04-03 00:00:00
519299
2021-04-03 00:00:00
605200
2021-04-03 00:00:00
570256
2021-04-03 00:00:00
546990
2021-04-03 00:00:00
641583
2021-04-03 00:00:00
668804
2021-04-03 00:00:00
592567
2021-04-03 00:00:00
623520
2021-04-03 00:00:00
622110
2021-04-03 00:00:00
570481
2021-04-03 00:00:00
594694
2021-04-03 00:00:00
663586
2021-04-03 00:00:00
666120
2021-04-03 00:00:00
660670
2021-04-03 00:00:00
645277
2021-04-03 00:00:00
518692
2021-04-03 00:00:00
542303
2021-04-03 00:00:00
544369
2021-04-03 00:00:00
516416
2021-04-03 00:00:00
621020
2021-04-03 00:00:00
665506
2021-04-03 00:00:00
656514
2021-04-03 00:00:00
621107
2021-04-03 00:00:00
457705
2021-04-03 00:00:00
542255
2021-04-03 00:00:00
518595
2021-04-03 00:00:00
595284
2021-04-03 00:00:00
592663
2021-04-03 00:00:00
596451
2021-04-03 00:00:00
622666
2021-04-03 00:00:00
656555
2021-04-03 00:00:00
547180
2021-04-03 00:00:00
664761
2021-04-03 00:00:00
543543
2021-04-03 00:00:00
641313
2021-04-03 00:00:00
594809
2021-04-03 00:00:00
547989
2

2021-04-04 00:00:00
643446
2021-04-04 00:00:00
656514
2021-04-04 00:00:00
543543
2021-04-04 00:00:00
547180
2021-04-04 00:00:00
592663
2021-04-04 00:00:00
516416
2021-04-04 00:00:00
542340
2021-04-04 00:00:00
607680
2021-04-04 00:00:00
596019
2021-04-04 00:00:00
666182
2021-04-04 00:00:00
665489
2021-04-04 00:00:00
543760
2021-04-04 00:00:00
624415
2021-04-04 00:00:00
643396
2021-04-04 00:00:00
663993
2021-04-04 00:00:00
545341
2021-04-04 00:00:00
643376
2021-04-04 00:00:00
606192
2021-04-04 00:00:00
624431
2021-04-04 00:00:00
642133
2021-04-04 00:00:00
672386
2021-04-04 00:00:00
642201
2021-04-04 00:00:00
518586
2021-04-04 00:00:00
621311
2021-04-04 00:00:00
608336
2021-04-04 00:00:00
641505
2021-04-04 00:00:00
669256
2021-04-04 00:00:00
571788
2021-04-04 00:00:00
665750
2021-04-04 00:00:00
593160
2021-04-04 00:00:00
643217
2021-04-04 00:00:00
642708
2021-04-04 00:00:00
593643
2021-04-04 00:00:00
614177
2021-04-04 00:00:00
644374
2021-04-04 00:00:00
656669
2021-04-04 00:00:00
643436
2

...

2021-06-25 00:00:00
467793
2021-06-25 00:00:00
521692
2021-06-25 00:00:00
643396
2021-06-25 00:00:00
663993
2021-06-25 00:00:00
656811
2021-06-25 00:00:00
642721
2021-06-25 00:00:00
624585
2021-06-25 00:00:00
571788
2021-06-25 00:00:00
666969
2021-06-25 00:00:00
669256
2021-06-25 00:00:00
641531
2021-06-25 00:00:00
608336
2021-06-25 00:00:00
642201
2021-06-25 00:00:00
628451
2021-06-25 00:00:00
593643
2021-06-25 00:00:00
656555
2021-06-25 00:00:00
596019
2021-06-25 00:00:00
624424
2021-06-25 00:00:00
624413
2021-06-25 00:00:00
592663
2021-06-25 00:00:00
457705
2021-06-25 00:00:00
663897
2021-06-25 00:00:00
665155
2021-06-25 00:00:00
642086
2021-06-25 00:00:00
606299
2021-06-25 00:00:00
546991
2021-06-25 00:00:00
547180
2021-06-25 00:00:00
594798
2021-06-25 00:00:00
546318
2021-06-25 00:00:00
543510
2021-06-25 00:00:00
591720
2021-06-25 00:00:00
621107
2021-06-25 00:00:00
643446
2021-06-25 00:00:00
607680
2021-06-25 00:00:00
641645
2021-06-25 00:00:00
608671
2021-06-25 00:00:00
663905
2

2021-06-26 00:00:00
641343
2021-06-26 00:00:00
664238
2021-06-26 00:00:00
641584
2021-06-26 00:00:00
518735
2021-06-26 00:00:00
622268
2021-06-26 00:00:00
643418
2021-06-26 00:00:00
547989
2021-06-26 00:00:00
571745
2021-06-26 00:00:00
572122
2021-06-26 00:00:00
660162
2021-06-26 00:00:00
666211
2021-06-26 00:00:00
641487
2021-06-26 00:00:00
606213
2021-06-26 00:00:00
544725
2021-06-26 00:00:00
670768
2021-06-26 00:00:00
641313
2021-06-26 00:00:00
664034
2021-06-26 00:00:00
641343
2021-06-26 00:00:00
643418
2021-06-26 00:00:00
571718
2021-06-26 00:00:00
664901
2021-06-26 00:00:00
664238
2021-06-26 00:00:00
683734
2021-06-26 00:00:00
620443
2021-06-26 00:00:00
641584
2021-06-26 00:00:00
518735
2021-06-26 00:00:00
571875
2021-06-26 00:00:00
641645
2021-06-26 00:00:00
656555
2021-06-26 00:00:00
664761
2021-06-26 00:00:00
554430
2021-06-26 00:00:00
663905
2021-06-26 00:00:00
643446
2021-06-26 00:00:00
546318
2021-06-26 00:00:00
592663
2021-06-26 00:00:00
624424
2021-06-26 00:00:00
543510
2

2021-06-27 00:00:00
663897
2021-06-27 00:00:00
502273
2021-06-27 00:00:00
489119
2021-06-27 00:00:00
546318
2021-06-27 00:00:00
457705
2021-06-27 00:00:00
664761
2021-06-27 00:00:00
591720
2021-06-27 00:00:00
663559
2021-06-27 00:00:00
656555
2021-06-27 00:00:00
673451
2021-06-27 00:00:00
664670
2021-06-27 00:00:00
606157
2021-06-27 00:00:00
592206
2021-06-27 00:00:00
458015
2021-06-27 00:00:00
543543
2021-06-27 00:00:00
663886
2021-06-27 00:00:00
571657
2021-06-27 00:00:00
663697
2021-06-27 00:00:00
663574
2021-06-27 00:00:00
643446
2021-06-27 00:00:00
656941
2021-06-27 00:00:00
607208
2021-06-27 00:00:00
665742
2021-06-27 00:00:00
467827
2021-06-27 00:00:00
664850
2021-06-27 00:00:00
543281
2021-06-27 00:00:00
502179
2021-06-27 00:00:00
595191
2021-06-27 00:00:00
605137
2021-06-27 00:00:00
543228
2021-06-27 00:00:00
516770
2021-06-27 00:00:00
621348
2021-06-27 00:00:00
641645
2021-06-27 00:00:00
606299
2021-06-27 00:00:00
642086
2021-06-27 00:00:00
624413
2021-06-27 00:00:00
641856
2

2021-06-28 00:00:00
656976
2021-06-28 00:00:00
641933
2021-06-28 00:00:00
425877
2021-06-28 00:00:00
444482
2021-06-28 00:00:00
642165
2021-06-28 00:00:00
657557
2021-06-28 00:00:00
663457
2021-06-28 00:00:00
662139
2021-06-28 00:00:00
670370
2021-06-28 00:00:00
607680
2021-06-28 00:00:00
663586
2021-06-28 00:00:00
621020
2021-06-28 00:00:00
501659
2021-06-28 00:00:00
661388
2021-06-28 00:00:00
656731
2021-06-28 00:00:00
450203
2021-06-28 00:00:00
624424
2021-06-28 00:00:00
624413
2021-06-28 00:00:00
628338
2021-06-28 00:00:00
641645
2021-06-28 00:00:00
660670
2021-06-28 00:00:00
518692
2021-06-28 00:00:00
645277
2021-06-28 00:00:00
643446
2021-06-28 00:00:00
642086
2021-06-28 00:00:00
543510
2021-06-28 00:00:00
606299
2021-06-28 00:00:00
596019
2021-06-28 00:00:00
546991
2021-06-28 00:00:00
501303
2021-06-28 00:00:00
467055
2021-06-29 00:00:00
643265
2021-06-29 00:00:00
546318
2021-06-29 00:00:00
605412
2021-06-29 00:00:00
543543
2021-06-29 00:00:00
665155
2021-06-29 00:00:00
656548
2

2021-06-29 00:00:00
642736
2021-06-29 00:00:00
602104
2021-06-29 00:00:00
488726
2021-06-29 00:00:00
493329
2021-06-29 00:00:00
670541
2021-06-29 00:00:00
621043
2021-06-29 00:00:00
663656
2021-06-29 00:00:00
596825
2021-06-29 00:00:00
656775
2021-06-29 00:00:00
669720
2021-06-29 00:00:00
514888
2021-06-29 00:00:00
623993
2021-06-29 00:00:00
624428
2021-06-29 00:00:00
668804
2021-06-29 00:00:00
595943
2021-06-29 00:00:00
621028
2021-06-29 00:00:00
605421
2021-06-29 00:00:00
663898
2021-06-29 00:00:00
658069
2021-06-29 00:00:00
592351
2021-06-29 00:00:00
641771
2021-06-29 00:00:00
663647
2021-06-29 00:00:00
592325
2021-06-29 00:00:00
543068
2021-06-29 00:00:00
641857
2021-06-29 00:00:00
624513
2021-06-29 00:00:00
572008
2021-06-29 00:00:00
606132
2021-06-29 00:00:00
641658
2021-06-29 00:00:00
453568
2021-06-29 00:00:00
570256
2021-06-29 00:00:00
602074
2021-06-29 00:00:00
664789
2021-06-29 00:00:00
570481
2021-06-29 00:00:00
650489
2021-06-29 00:00:00
605548
2021-06-29 00:00:00
514917
2

In [86]:
df_final = pd.merge(
    batting_stats,
    df_wpitching,
    how="right",
    on=None,
    left_on=None,
    right_on=None,
    left_index=True,
    right_index=True,
    sort=False,
    suffixes=("_bs", "_df"),
    copy=True,
    indicator=False,
    validate=None,
)
#

In [87]:
df_final = df_final.rename(columns={'games_played_last_2_years_df': 'games_played_last_2_years_pitcher', 'games_played_last_2_years_bs': 'games_played_last_2_years_batter'})

#     transforms date and datetime columns into easily comparable numbers
df_final['local_datetime'] = pd.to_datetime(df_final['local_datetime'])
df_final['local_date'] = df_final['local_datetime'].apply(lambda x: x.toordinal())
df_final['local_datetime'] = df_final['local_datetime'].apply(lambda x: x.timestamp())

#     drop columns that are functionally duplicates, contain information that wouldn't be known prior to game,
#     or are irrelevant


drop_columns= ['player_name',
 'game_type_dff', 'pitch_type',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'pitcher',
 'events',
 'description',
 'spin_dir',
'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'des',
  'p_throws',
'type',
 'hit_location',
 'bb_type',
 'balls',
 'strikes',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
  'sv_id',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'pitch_name',
 'home_score_dff',
 'away_score_dff',
 'bat_score',
 'fld_score',
 'post_away_score',
 'post_home_score',
 'post_bat_score',
 'post_fld_score',
 'if_fielding_alignment',
 'of_fielding_alignment',
 'spin_axis',
 'delta_home_win_exp',
 'delta_run_exp',
 'game_pk',
 'game_id','game_date_gaw_p',
 'game_datetime',
 'game_date_gaw_df',
 'game_type_gaw',
 'datetime',
#  'datetimeStr',
 'game_date_dff','winning_team',
 'losing_team',
 'winning_pitcher',
 'losing_pitcher',
 'save_pitcher',
 'summary', 'home_probable_pitcher',
 'away_probable_pitcher',
 'home_pitcher_note',
 'away_pitcher_note',
 'away_score_gaw',
 'home_score_gaw',
 'current_inning',
 'inning_state',
 'venue_id',
 'status',
 'home_team', 'away_team', 'home_id', 'away_id', 'pitcher_righty',
   'game_type',
  'solarenergy', 'solarradiation', 'snow', 'precipcover', 'info',
              'maxt', 'mint', 'weathertype']

for col in df_final.columns:
    if col in drop_columns:
        df_final = df_final.drop(columns=[col])

In [89]:
def evaluate(estimator, X_train, X_val, y_train, y_val, holdout, roc_auc='proba', output = False):

    X_holdout = holdout.drop(['got_a_hit'], axis=1)
    y_holdout = holdout['got_a_hit']
    
    #     grab predictions
    train_preds = estimator.predict(X_train)
    val_preds = estimator.predict(X_val)
    holdout_preds = estimator.predict(X_holdout)
    
#     output needed for roc_auc score
    if roc_auc == 'skip':
        train_out = False
        val_out = False
        holdout_out = False
    elif roc_auc == 'dec': # not all classifiers have a decision function
        train_out = estimator.decision_function(X_train)
        val_out = estimator.decision_function(X_val)
        holdout_out = estimator.decision_function(X_holdout)
    elif roc_auc == 'proba':
        try:
            train_out = estimator.predict_proba(X_train)[:, 1]
            val_out = estimator.predict_proba(X_val)[:, 1]
            holdout_out = estimator.predict_proba(X_holdout)[:, 1]
        except AttributeError:
            train_out = estimator.predict(X_train)
            val_out = estimator.predict(X_val)
            holdout_out = estimator.predict(X_holdout)

    else:
        raise Exception("The value for roc_auc should be 'skip', 'dec', or 'proba'.")
    
    ac = accuracy_score(y_train, train_preds)
    f1 = f1_score(y_train, train_preds)
    ras = roc_auc_score(y_train, train_out)
    pr = precision_score(y_train, train_preds)
    
    if output == True:
        print('Train Scores')
        print('------------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(train_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

    ac = accuracy_score(y_val, val_preds)
    f1 = f1_score(y_val, val_preds)
#     print(type(y_val))
    ras = roc_auc_score(y_val, val_out)
    pr = precision_score(y_val, val_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Val Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(val_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')
    
    ac = accuracy_score(y_holdout, holdout_preds)
    f1 = f1_score(y_holdout, holdout_preds)
#     print(type(holdout['got_a_hit']))
    ras = roc_auc_score(y_holdout, holdout_out)
    pr = precision_score(y_holdout, holdout_preds)
    
    if output == True:
        print('-----------------------------------')
        print('Holdout Scores')
        print('-----------')
        print(f'Accuracy: {ac}')
        print(f'F1 Score: {f1}')
        if type(holdout_out) == np.ndarray:
            print(f'ROC-AUC: {ras}')
        print(f'Precision: {pr}')

        print('\nVal Data')
        print('-----------')

        plot_confusion_matrix(estimator, X_val, y_val, values_format=',.5g')
        plt.show()
        
        print('Holdout Data')        
        print('-----------')

        plot_confusion_matrix(estimator, X_holdout, y_holdout, values_format=',.5g')
        plt.show()

In [90]:
def streak_checker(dataframe, output = False, metric = 'odds', ascending=False, combiner=None, timed=False, today=False):
    results = []

    try:
        if 'L/R_split' in metric:
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy = dataframe.copy()
                dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] + row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['batter_righty'] == 0) else (row['H/PA_against_L'] + row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['batter_righty'] == '1') else 0), axis=1)
                odds_scaler(dataframe_copy, timed, start)
                dataframe = dataframe_copy
                metric='temp'
            elif combiner == 'multiply':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy = dataframe.copy()
                dataframe_copy['temp'] = dataframe.apply(lambda row: row['H/PA_against_R'] * row['against_lefties_H/PA'] if (row['pitch_hand'] == 'Right' and row['batter_righty'] == 0) else (row['H/PA_against_L'] * row['against_righties_H/PA'] if (row['pitch_hand'] == 'Left' and row['batter_righty'] == 1) else 0), axis=1)
                odds_scaler(dataframe_copy, timed, start)
                dataframe = dataframe_copy
                metric='temp'
        if type(metric) == list:
            dataframe_copy = dataframe.copy()
            if combiner == 'add':
                if timed == True:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] + row[metric[1]], axis=1)
                odds_scaler(dataframe_copy, timed, start)
            elif combiner == 'multiply':
                if timed:
                    start = datetime.now()

                    print(combiner, start)
                dataframe_copy['temp'] = dataframe.apply(lambda row: row[metric[0]] * row[metric[1]], axis=1)
                odds_scaler(dataframe_copy, timed, start)
            else:
                raise KeyError(f'{combiner} combiners have not been implemented yet')
            dataframe = dataframe_copy
            metric = 'temp'
    except KeyError:
        raise KeyError("metric must be chosen from dataframe's columns")

    date_list = np.sort(dataframe['local_date'].unique())

    for day in date_list:
        best_bet = dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending).iloc[0]
#         return(dataframe[dataframe['local_date'] == day].sort_values(metric, ascending=ascending))
        odds = best_bet[metric]
        date = datetime.fromordinal(best_bet['local_date']).strftime("%b %d %Y")

        player_info = pybaseball.playerid_reverse_lookup([best_bet['batter']])
        if len(player_info) >1:
            player_info = player_info.iloc[0]
            player = player_info['key_mlbam']
        if best_bet['home'] == 0:
            team = best_bet['away_name']
        else:
            team = best_bet['home_name']
    #     team = statsapi.lookup_team(player['currentTeam']['id'])[0]['name']
    #     player_name = player['fullName']
        if today:
#             actual_result = None
            return player.title() + ' - ' + team
        else:
            actual_result = best_bet['got_a_hit']
        results.append(actual_result)
        if output == True and len(date_list) > 20 and day > date_list[-20]:
            print(date, player, team, odds, actual_result)
#                 print(date, player, team, odds)
    longest = 0
    current = 0
    for num in results:
        if num == 1:
            current += 1
        else:
            longest = max(longest, current)
            current = 0
    if output == True and today == False:
        print(f'Longest streak in set ({len(results)} days): ', longest)
        print(f'Total correct guesses in set ({len(results)} days): ', sum(results), sum(results)/len(results))

def odds_scaler(dataframe_copy, timed, start):
    mms = MinMaxScaler()
    dataframe_copy['temp'] = pd.DataFrame(mms.fit_transform(dataframe_copy[['temp']]))

    if timed:
        print(datetime.now() - start)

In [91]:
def run_model(clf, X_train, X_val, y_train, y_val, holdout):

    probability = True
    
    clf.fit(X_train.head(100), y_train.head(100))

    evaluate(clf, X_train.head(100), X_val.head(100), y_train.head(100), y_val.head(100), holdout);

    runtime = datetime.now() - start

    if runtime > timedelta(seconds=5):
        print(f'Model would take {runtime.total_seconds()*200/60} minutes to run.')
    else:
        print('Model should take between {:.0f} and {:.0f} minutes to run and finish by {}'.format(runtime.total_seconds()*20/60, runtime.total_seconds()*200/60, (datetime.now()+(timedelta(seconds=(runtime.total_seconds()*200)))).strftime("%H:%M")))
        clf.fit(X_train, y_train)
        
        with open('models/'+list(clf.named_steps.keys())[1], 'wb') as f:
            pickle.dump(clf, f, protocol=pickle.HIGHEST_PROTOCOL)
            
        evaluate(clf, X_train, X_val, y_train, y_val, holdout, output=True)
        plot_roc_curve(clf, X_val, y_val)
        plt.show()
        
        try:
            val_df_odds = pd.Series(clf.predict_proba(X_val)[:, 1], name='odds')
            holdout_df_odds = pd.Series(clf.predict_proba(holdout.drop(['got_a_hit'], axis=1))[:, 1], name='odds')
        except AttributeError:
            probability = False
            val_df_odds = pd.Series(clf.predict(X_val), name= 'odds')
            holdout_df_odds = pd.Series(clf.predict(holdout.drop(['got_a_hit'], axis=1)), name='odds')

        val_df = X_val.assign(got_a_hit = y_val).reset_index(drop=True)
        val_df = val_df.assign(odds=val_df_odds)
        holdout = holdout.reset_index(drop=True)
        holdout = holdout.assign(odds=holdout_df_odds)
#         return(holdout)

# This check is to prevent returning recommendations based solely on models that do not return probabilities. However, it appears that randomly selected 1s may perform better than sorted 1s once passed through some filter such as plate appearances per game

#         TODO change to .issubset()
#         if set(val_df_odds.value_counts().index) == set([1, 0, 1.0, 0.0]):
#             print(str(clf.get_params()['steps'][1][1]), "doesn't return probabilities, so no streak results will be returned based on this alone.")
#             pass
#         else:
#             print()
        print('-----------------------------------')

        print('Val data')
        print('-----------')

        streak_checker(val_df, output=True)
        print('Holdout data')
        print('-----------')

        streak_checker(holdout, output=True)
            
        print('Odds multiplied by PA/G_batter')
        print('-----------')
        
        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
        print('-----------')
        print('Odds added to PA/G_batter')
        print('-----------')

        print('Val data')
        print('-----------')
        streak_checker(val_df, output=True, metric=['L/R_split', 'odds'], combiner='add')
        print('-----------')
        print('Holdout data')
        print('-----------')
        streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')

In [104]:
def train_test_split(df, test_size=.25):
    
    '''My simplified train_test_split which just cuts the dataframe by date, in order to accurately evaluate streak results, to roughly the test size requested'''
    
    sample_size = len(df)
    
    train_size = 1 - test_size
    
    train_size = round(train_size * sample_size)
    
    df = df.sort_values(['game_date']).reset_index(drop=True)
    
    date_cutoff = df.iloc[train_size]['game_date']

    train = df[df['game_date'] <= date_cutoff]
    val = df[df['game_date'] > date_cutoff]
    
    X_train = train.drop(['got_a_hit'], axis=1)
    y_train = train['got_a_hit']
    
    X_val = val.drop(['got_a_hit'], axis=1)
    y_val = val['got_a_hit']
    
    return(X_train, X_val, y_train, y_val)

In [97]:
df_final = pd.read_csv('df_final.csv', index_col=0)

In [98]:
df_final.columns.to_list()

['game_date',
 'release_speed',
 'release_pos_x',
 'release_pos_z',
 'batter',
 'pitcher',
 'events',
 'spin_dir',
 'spin_rate_deprecated',
 'break_angle_deprecated',
 'break_length_deprecated',
 'zone',
 'hit_location',
 'balls',
 'strikes',
 'game_year',
 'pfx_x',
 'pfx_z',
 'plate_x',
 'plate_z',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'inning',
 'hc_x',
 'hc_y',
 'tfs_deprecated',
 'tfs_zulu_deprecated',
 'fielder_2',
 'umpire',
 'vx0',
 'vy0',
 'vz0',
 'ax',
 'ay',
 'az',
 'sz_top',
 'sz_bot',
 'hit_distance_sc',
 'launch_speed',
 'launch_angle',
 'effective_speed',
 'release_spin_rate',
 'release_extension',
 'game_pk',
 'pitcher.1',
 'fielder_2.1',
 'fielder_3',
 'fielder_4',
 'fielder_5',
 'fielder_6',
 'fielder_7',
 'fielder_8',
 'fielder_9',
 'release_pos_y',
 'estimated_ba_using_speedangle',
 'estimated_woba_using_speedangle',
 'woba_value',
 'woba_denom',
 'babip_value',
 'iso_value',
 'launch_speed_angle',
 'at_bat_number',
 'pitch_number',
 'home_score',
 'away_sc

In [99]:
df_final

,game_date,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,hit_location,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,batter_righty,pitcher_righty,home,sv_id
0,2015-10-04,97,-1.08,6.20,150029,544727,0,NaN,NaN,NaN,NaN,7,8,0,1,2015,-1.12,0.46,-0.71,2.02,501571,547180,491696,2,9,110.32,71.66,NaN,NaN,518595,NaN,3.600000,-141.270000,-6.540000,-14.350000,32.150000,-28.510000,3.64,1.67,345,95.2,35.0,96.5,2018,6.1,416079,20439,-5693,-12489,-21771,-27601,-9375,-30936,-22717,-24594,50.00,0.153,0.293,0,1,0,0.0,3.0,61,2,1,0,0,1,0,1,0,1,NaN,0.112,-0.256,1,1,0,NaN
1,2015-10-04,97,-0.93,6.42,150029,544727,-1,NaN,NaN,NaN,NaN,3,5,0,0,2015,-1.11,0.71,0.44,3.50,501571,547180,491696,2,9,NaN,NaN,NaN,NaN,518595,NaN,6.390000,-141.720000,-3.750000,-14.310000,32.100000,-24.840000,3.64,1.67,180,NaN,NaN,96.5,2093,6.0,416079,20439,-5693,-12489,-21771,-27601,-9375,-30936,-22717,-24594,50.00,NaN,NaN,0,1,0,NaN,NaN,61,1,1,0,0,1,0,1,0,1,NaN,0.000,-0.048,1,1,0,NaN
2,2015-10-04,98,-1.01,6.21,547180,544727,1,NaN,NaN,NaN,NaN,4,7,0,0,2015,-1.52,0.52,-0.35,2.45,501571,493472,491696,2,9,73.19,139.24,NaN,NaN,518595,NaN,5.480000,-143.010000,-5.670000,-20.490000,36.600000,-27.460000,3.19,1.46,20,72.2,-6.0,97.6,1960,6.4,416079,20439,-5693,-12489,-21771,-27601,-9375,-30936,-22717,-24594,50.00,0.093,0.084,1,1,1,1.0,2.0,60,1,1,0,0,1,0,1,0,1,NaN,-0.077,0.208,0,1,0,NaN
3,2015-10-04,97,-0.96,6.08,607208,544727,0,NaN,NaN,NaN,NaN,13,2,3,2,2015,-1.29,0.62,-0.05,1.27,501571,493472,491696,1,9,NaN,NaN,NaN,NaN,518595,NaN,5.580000,-141.820000,-8.800000,-16.940000,31.900000,-26.150000,3.50,1.61,180,NaN,NaN,97.1,2099,6.3,416079,20439,-5693,-12489,-21771,-27601,-9375,-30936,-22717,-24594,50.00,NaN,NaN,0,1,0,0.0,NaN,59,7,1,0,0,1,0,1,0,1,NaN,0.052,-0.200,1,1,0,NaN
4,2015-10-04,98,-1.02,6.09,607208,544727,-1,NaN,NaN,NaN,NaN,7,5,3,2,2015,-1.00,0.80,-0.41,1.56,501571,493472,491696,1,9,NaN,NaN,NaN,NaN,518595,NaN,3.990000,-142.750000,-8.510000,-12.710000,36.270000,-23.520000,3.50,1.61,180,NaN,NaN,97.7,2155,6.5,416079,20439,-5693,-12489,-21771,-27601,-9375,-30936,-22717,-24594,50.00,NaN,NaN,0,1,0,NaN,NaN,59,6,1,0,0,1,0,1,0,1,NaN,0.000,0.000,1,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4113589,2021-07-01,86,3.36,5.77,643396,640455,0,NaN,NaN,NaN,NaN,8,5,1,2,2021,1.17,0.45,0.26,1.81,608070,608070,607680,0,1,95.59,160.50,NaN,NaN,605244,NaN,-9.921421,-125.599164,-4.153304,14.419476,25.764913,-26.781086,3.32,1.51,5,74.8,-20.0,87.5,1583,7.1,633405,-14905,15420,31742,17952,945,3349,-11967,2296,9553,53.41,0.060,0.060,0,1,0,0.0,2.0,1,5,0,0,0,0,0,0,0,0,120.0,0.022,-0.174,1,0,0,NaN
4113590,2021-07-01,91,3.53,5.54,643396,640455,-1,NaN,NaN,NaN,NaN,12,5,0,2,2021,1.46,1.20,0.40,4.57,608070,608070,607680,0,1,NaN,NaN,NaN,NaN,605244,NaN,-11.231747,-133.250744,0.901543,19.932732,32.151844,-18.454876,3.44,1.54,164,NaN,NaN,92.5,1999,7.1,633405,-14905,15420,31742,17952,945,3349,-11967,2296,9553,53.36,NaN,NaN,0,1,0,NaN,NaN,1,4,0,0,0,0,0,0,0,0,138.0,0.000,0.023,1,0,0,NaN
4113591,2021-07-01,92,3.39,5.49,6

In [105]:
X_train, X_val, y_train, y_val = train_test_split(df_final)

KeyError: "['got_a_hit'] not found in axis"

In [ ]:
num_cols = [] # all numeric columns
cols_to_ohe = [] # doubleheader, conditions, batter_righty
cols_to_targ = [] # all other object columns

for c in X_train.columns:
    if X_train[c].dtype in ['float64', 'int64'] and c not in ['game_num', 'batter', 'fielder_2', 'umpire']:
        num_cols.append(c)
    elif len(X_train[c].unique()) < 10:
        cols_to_ohe.append(c)
    else:
        cols_to_targ.append(c)

In [ ]:
num_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

ohe_transformer = Pipeline(steps=[
    ('ohe_imputer', SimpleImputer(strategy='constant', fill_value='Unknown')),
    ('ohencoder', OneHotEncoder(handle_unknown='ignore'))
])

target_encoder = Pipeline(steps=[
    ('freq_enc', ce.target_encoder.TargetEncoder()),
    ('freq_imputer', SimpleImputer(strategy='constant', fill_value=0))
    
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_cols),
        ('ohe', ohe_transformer, cols_to_ohe),
        ('target', target_encoder, cols_to_targ)
    ])

# Baseline - dummy models
Pick player with the highest Plate Appearances per Game over the last 2 years

In [ ]:
streak_checker(df, output=True, metric='PA/G_batter')
streak_checker(test_df, output=True, metric='PA/G_batter')

Observations - this seems to trend towards a small group of players except for when a player makes their mLB debut towards the top of the lineup (Wander Franco, Greg Deichmann). Sample size is an issue - could be addressed by either looking at games played or excluding PA/G above 4.9.



Pick player with the highest launch speed

In [ ]:
streak_checker(df, output=True, metric='avg_launch_speed')
streak_checker(test_df, output=True, metric='avg_launch_speed')

The metrics point to this being mostly useless which is backed up by the fact that it seems to occasionally choose pitchers (Hunter Strickland, Zack Greinke), when it's not fawning over Aaron Judge. Could be worth excluding pitchers as a class though would be hard to implement with the current dataset, not to mention the fact that 2-way players would need to be excluded from that filter.

In [ ]:
streak_checker(df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')
streak_checker(test_df, output=True, metric=['avg_launch_angle', 'avg_launch_speed'], combiner='add')

Exaserbates bias towards small sample sizes, leading part time players to dominate selection rather than proven hitters

Would also be worth checking lefty righty splits

In [ ]:
streak_checker(df, output=True, metric='L/R_split', combiner='add')
streak_checker(test_df, output=True, metric='L/R_split', combiner='add')
streak_checker(df, output=True, metric='L/R_split', combiner='multiply')
streak_checker(test_df, output=True, metric='L/R_split', combiner='multiply')

Nothing stands out in these groups. Seems to be a mix of journeymen and all-stars.

# Model 1 - Decision Tree

In [ ]:
start = datetime.now()
dt = DecisionTreeClassifier()

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('tree', dt)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 2: Logistic Regression

In [ ]:
start = datetime.now()

logreg = LogisticRegression(solver='sag')

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('logreg', logreg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3: SVC

In [ ]:
start = datetime.now()

svc = SVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('svc', svc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 3.5: LinearSVC

In [ ]:
start = datetime.now()

Linearsvc = LinearSVC()


clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('linearsvc', Linearsvc)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 4: KNN

In [ ]:
start = datetime.now()

knn = KNeighborsClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('knn', knn)
])


run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 5: Random Forest

In [ ]:
start = datetime.now()

rfc = RandomForestClassifier(n_jobs=-1)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rfc', rfc)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 6: AdaBoost


In [ ]:
start = datetime.now()

ada = AdaBoostClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('ada', ada)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 7: GradientBoost


In [ ]:
start = datetime.now()

gbm = GradientBoostingClassifier()
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('gbm', gbm)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 8: XGBoost


In [ ]:
start = datetime.now()

xbg = xgb.XGBClassifier(random_state=42)

clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('xbg', xbg)
])

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Model 9: Random Forest with Grid Search


In [ ]:
start = datetime.now()

parameters = {
    'min_samples_split': [3, 5, 100], 
    'n_estimators' : [100, 300],
        'max_depth': [3, 5, 15, 25],

}

run_model(clf, X_train, X_val, y_train, y_val, test_df)

# Daily Guess

In [462]:
def time_date_weather_data(game_pk_df):
    venue_coords = pd.read_csv('Parks.csv')
    data_collection = {}

    game_pk_df['coordinates'] = game_pk_df['venue_name'].apply(lambda x: ','.join([str(venue_coords[venue_coords['NAME'] == x]['Latitude'].iloc[0]), str(venue_coords[venue_coords['NAME'] == x]['Longitude'].iloc[0])]))
    game_pk_df['alt'] = game_pk_df['venue_name'].apply(lambda x: venue_coords[venue_coords['NAME'] == x]['Altitude'].iloc[0])


    game_pk_df = game_pk_df.sort_values(['game_datetime'])



    game_pk_df['local_datetime'] = game_pk_df.apply(lambda row: convert_UTC_to_local(row), axis=1)
    game_pk_df['local_datetime'] = game_pk_df['local_datetime'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))


    from api_keys import visual_crossing
    import requests

    weather_df = pd.DataFrame()
    for index, row in game_pk_df.iterrows():
#         check if key is past or future
        if datetime.strptime(row['game_datetime'], "%Y-%m-%dT%H:%M:%SZ") > datetime.utcnow():
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'

        else:
            URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?&aggregateHours=1&startDateTime={row["local_datetime"]}&endDateTime={row["local_datetime"]}&unitGroup=us&contentType=json&location={row["coordinates"]}&key={visual_crossing}'
        global api_counter
        if api_counter >= 1000:
            print('Prepreprocessing will continue at ' + (datetime.now()+timedelta(seconds = 86400)).strftime('%H:%M:%S'))
            sleep(86400)
            api_counter = 0
            api_counter += 1
            response = requests.get(URL)
        try:
            data = response.json()
            if (
                data['locations'][row['coordinates']]['values'][0]['temp']
                is None
            ):
                URL = f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/forecast?locations={row["coordinates"]}&aggregateHours=1&unitGroup=us&forecastDays=1&shortColumnNames=true&contentType=json&key={visual_crossing}'
                api_counter += 1
                response = requests.get(URL)
                data = response.json()

            if len(data['locations'][row['coordinates']]['values']) > 1:
                for forecast in data['locations'][row['coordinates']]['values']:

                    if forecast['datetimeStr'][:13] == row['local_datetime'][:13]:
                        values = forecast
                        values['datetimeStr'] = row['local_datetime']


            else:
                values = data['locations'][row['coordinates']]['values'][0]
                values['datetimeStr'] = row['local_datetime']
            data_collection[URL] = data
            weather_df = weather_df.append({**{'coordinates': row['coordinates']}, **values}, ignore_index=True)

        except:
            raise IndexError(index, row, response, response.json())
    weather_df = weather_df.rename(columns={'datetimeStr': 'local_datetime'})


    return pd.merge(
        game_pk_df,
        weather_df,
        how="left",
        on=None,
        left_on=['local_datetime', 'coordinates'],
        right_on=['local_datetime', 'coordinates'],
        left_index=False,
        right_index=False,
        sort=False,
        suffixes=("_gpk", "_wdf"),
        copy=True,
        indicator=False,
        validate=None,
    )

In [463]:
def lookup_mlb():
    """Get list of MLB team IDs for the purposes of pulling active rosters"""
    params = {
        "activeStatus": 'Y',
        "season": 2021,
        "sportIds": 1,
        "fields": "teams,id,name,league",
    }

    r= statsapi.get('teams', params)

    return {
        team['name']: team['id']
        for team in r['teams']
        if team['league']['id'] in [103, 104]
    }

def roster(teamId, rosterType=None, season=datetime.now().year, date=None):
    """Get the roster for a given team."""
    if not rosterType:
        rosterType = "active"

    params = {"rosterType": rosterType, "season": season, "teamId": teamId}
    if date:
        params.update({"date": date})

    r = statsapi.get("team_roster", params)

    return [x["person"]["id"] for x in r["roster"]]

def lookup_team(lookup_value, activeStatus="Y", season=datetime.now().year, sportIds=1):
    """Get a info about a team or teams based on the team name, city, abbreviation, or file code."""
    params = {
        "activeStatus": activeStatus,
        "season": season,
        "sportIds": sportIds,
        "fields": "teams,id,name,teamCode,fileCode,teamName,locationName,shortName,league",
    }
    r = statsapi.get("teams", params)

    teams = []
    for team in r["teams"]:
        for v in team.values():
            if str(lookup_value).lower() in str(v).lower():
                teams.append(team)
                break

    return (teams)

In [474]:
# to be run daily
def daily_guess():
    '''Implementing a daily recommendation engine based on the LinearSCV model'''
    # get game data, then weather data, then pitcher data, then batter data

    todays_games = pd.DataFrame()


    for game in statsapi.schedule():
    #     grab 'away_probable_pitcher','home_probable_pitcher','away_name',
    #     'home_name','doubleheader','game_num','venue_name','game_datetime'

    #     df['local_datetime'] = pd.to_datetime(df['local_datetime'])
    #     df['local_date'] = df['local_datetime'].apply(lambda x: x.toordinal())
    #     df['local_datetime'] = df['local_datetime'].apply(lambda x: x.timestamp())

        row = {
            'game_id': game['game_id'],
            'away_probable_pitcher': game['away_probable_pitcher'],
            'home_probable_pitcher': game['home_probable_pitcher'],
            'away_name': game['away_name'],
            'home_name': game['home_name'],
            'doubleheader': game['doubleheader'],
            'game_num': game['game_num'],
            'venue_name': game['venue_name'],
            'game_datetime': game['game_datetime']
        }

        todays_games = todays_games.append(row, ignore_index=True)

    todays_games = todays_games.replace('', np.nan)    
    global games_and_weather
    games_and_weather = time_date_weather_data(todays_games)
# def daily_guess(games_and_weather):
    for i in ['home_probable_pitcher', 'away_probable_pitcher']:
        games_and_weather[i] = games_and_weather[i].fillna('Unknown')
    away_pitchers = games_and_weather['away_probable_pitcher']
    home_pitchers = games_and_weather['home_probable_pitcher']
    global pitchers_stats
    pitchers_stats = pd.DataFrame()

    history = pd.concat([
    set_types(pd.read_csv('data/2019_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2020_statcast_pbp.csv'), type_dict),
    set_types(pd.read_csv('data/2021_statcast_pbp.csv'), type_dict)
    ], ignore_index=True)
    
    sample = pd.read_csv('df_final', index_col=0)
    pitchers_stats_cols = ['pitch_hand', 'games_played_last_2_years_pitcher',
       'avg_innings_per_appearance', 'avg_PAs_per_appearance_pitcher',
       'avg_hits_per_inning', 'H/PA_pitcher', 'against_lefties_H/PA',
       'against_righties_H/PA', 'probable_pitcher']
    
    average_pitcher = {}

    for col in pitchers_stats_cols:
        if col == 'probable_pitcher':
            col_val = 'Unknown'
        else:
            try:
                col_val = sample[col].mean()
            except TypeError:
                col_val = list(sample[col].value_counts().index)[0]
        average_pitcher[col] = col_val
    
    
    for a in away_pitchers:
        print(a)
        if a == 'Unknown':
            pitchers_stats = pitchers_stats.append(pd.Series(average_pitcher), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats(a, 'pitcher', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'probable_pitcher': a}}, ignore_index=True)

    for h in home_pitchers:
        print(h)
        if h == 'Unknown':
            pitchers_stats = pitchers_stats.append(pd.Series(average_pitcher), ignore_index=True)
        else:
            pitchers_stats = pitchers_stats.append({**derived_cumulative_stats(h, 'pitcher', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'probable_pitcher': h}}, ignore_index=True)

    game_columns = ['away_name',
                     'home_name',
                     'doubleheader',
                     'game_num',
                     'venue_name',
                     'local_datetime',
                     'coordinates',
                     'cloudcover',
                     'conditions',
                     'dew',
                     'heatindex',
                     'humidity',
                     'precip',
                     'sealevelpressure',
                     'snowdepth',
                     'temp',
                     'visibility',
                     'wdir',
                     'wgust',
                     'windchill',
                     'wspd',
                     'alt'
     ]
            
    batting_stats = pd.DataFrame()
    for index, game in games_and_weather.iterrows():
        print(game[['away_name', 'home_name']])
    #     home batters, away pitcher
        batting_team_id = lookup_mlb()[game['home_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            start = datetime.now()
            
            batter_righty = 1 if statsapi.player_stat_data(b)['bat_side'] == 'Right' else 0
            home = 1 if game['home_name'] == statsapi.player_stat_data(b)['current_team'] else 0
            batting_stats = batting_stats.append({**derived_cumulative_stats(b, 'batter', history, end_date=(datetime.now()-timedelta(seconds=86400))),**{'batter': b, 'game_year': datetime.now().year, 'batter_righty': batter_righty, 'home': home}, **pitchers_stats[pitchers_stats['probable_pitcher'] == game['away_probable_pitcher']].iloc[0], **game[game_columns]}, ignore_index=True)
            print(datetime.now()-start)
        #     away batters, home pitcher
        batting_team_id = lookup_mlb()[game['away_name']]
        batting_roster = roster(batting_team_id)
        for b in batting_roster:
            batter_righty = 1 if statsapi.player_stat_data(b)['bat_side'] == 'Right' else 0
            home = 1 if game['home_name'] == statsapi.player_stat_data(b)['current_team'] else 0
            batting_stats = batting_stats.append({**derived_cumulative_stats(b, 'batter', history, end_date=(datetime.now()-timedelta(seconds=86400))), **{'batter': b, 'game_year': datetime.now().year, 'batter_righty': batter_righty, 'home': home}, **pitchers_stats[pitchers_stats['probable_pitcher'] == game['home_probable_pitcher']].iloc[0], **game[game_columns]}, ignore_index=True)
            
    batting_stats['covid_doubleheader'] = batting_stats.apply(
    lambda row: 1 if row['game_year'] in [2020, 2021] and row['doubleheader'] == 'Y' else 0, axis=1)
    
    american_league_teams = np.array(['Boston Red Sox', 'Houston Astros', 'Chicago White Sox', 'Tampa Bay Rays', 'Oakland Athletics', 'Seattle Mariners', 'New York Yankees', 'Toronto Blue Jays', 'Los Angeles Angels', 'Cleveland Indians', 'Detroit Tigers', 'Kansas City Royals', 'Minnesota Twins', 'Texas Rangers', 'Baltimore Orioles'])
    
    batting_stats['designated_hitter'] = batting_stats.apply(
    lambda row: 1 if row['home_name'] in american_league_teams or row['game_year'] == 2020 else 0, axis=1)
    
        #     transforms date and datetime columns into easily comparable numbers
    batting_stats['local_datetime'] = pd.to_datetime(batting_stats['local_datetime'])
    batting_stats['local_date'] = batting_stats['local_datetime'].apply(lambda x: x.toordinal())
    batting_stats['local_datetime'] = batting_stats['local_datetime'].apply(lambda x: x.timestamp())
    
    return(batting_stats)
# TODO
# RUN batting_stats through predictors

In [475]:
def run_model_todays_data(batting_stats):
    recommendations = []
    for model in os.listdir('models/'):
        with open('models/' + model, 'rb') as f:
            clf = pickle.load(f)
            try:
                todays_odds = pd.Series(clf.predict_proba(batting_stats[X_train.columns])[:, 1], name='odds')
            except AttributeError:
                todays_odds = pd.Series(clf.predict(batting_stats[X_train.columns]), name='odds')

            batting_stats_temp = batting_stats.reset_index(drop=True)
            batting_stats_temp = batting_stats.assign(odds=todays_odds)
            
#             print('-----------------------------------')

#             print(model)
#             print('-----------')

            recommendations.append(streak_checker(batting_stats_temp, output=True, today=True))
#             print(recommendations)
#             print('Holdout data')
#             print('-----------')

#             streak_checker(holdout, output=True)
#             ('-----------------------------------')
#             print('\n\nOdds multiplied by PA/G_batter\n\n')
            
#             print('-----------')

#             print('Val data')
#             print('-----------')
            recommendations.append(streak_checker(batting_stats_temp, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply', today=True))
#             print('Holdout data')
#             print('-----------')
#             streak_checker(holdout, output=True, metric=['PA/G_batter', 'odds'], combiner='multiply')
#             print('-----------')
#             ('-----------------------------------')
#             print('\n\nOdds added to PA/G_batter\n\n')
            
#             print('-----------')

#             print('Val data')
#             print('-----------')
            recommendations.append(streak_checker(batting_stats_temp, output=True, metric=['L/R_split', 'odds'], combiner='add', today=True))
#             print('-----------')
#             print('Holdout data')
#             print('-----------')
#             streak_checker(holdout, output=True, metric=['L/R_split', 'odds'], combiner='add')
    return(recommendations)

In [476]:
start = datetime.now()

batting_stats = daily_guess()

datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
datetime done
events done
stand done
pitcher done
inning done
Triston McKenzie
2021-09-13 12:46:52.209895
Triston McKenzie
player filtered
Wade Miley
2021-09-13 12:46:53.036122
Wade Miley
player filtered
Freddy Peralta
2021-09-13 12:46:53.607314
Freddy Peralta
player filtered
Adrian Sampson
2021-09-13 12:46:54.112034
Adrian Sampson
player filtered
Jesus Luzardo
2021-09-13 12:46:54.566127
Jesus Luzardo
player filtered
Gerrit Cole
2021-09-13 12:46:55.065997
Gerrit Cole
player filtered
Drew Rasmussen
2021-09-13 12:46:55.601036
Drew Rasmussen
player filtered
Jake Woodford
2021-09-13 12:46:56.156067
Jake Woodford
player filtered
Jon Gray
2021-09-13 12:46:56.669261
Jon Gray
player filtered
Logan Allen
2021-09-13 12:46:57.152867
Logan Allen
player filtered
Zack Greinke
2021-09-13 12:46:57.668107
Zack Greinke
player filtered
Frankie Montas
2021-09-13 12:46:58.211849
Frank

2021-09-13 12:48:28.246272
666159
0:00:00.618723
2021-09-13 12:48:28.816762
605242
0:00:00.594198
2021-09-13 12:48:29.394046
408234
0:00:00.620696
2021-09-13 12:48:30.049362
592348
0:00:00.611616
2021-09-13 12:48:30.771007
543257
0:00:00.758077
2021-09-13 12:48:31.383177
669373
0:00:00.541238
2021-09-13 12:48:31.951713
641302
0:00:00.593817
2021-09-13 12:48:32.502751
622682
0:00:00.594415
2021-09-13 12:48:33.159685
650489
0:00:00.639316
2021-09-13 12:48:33.839228
503449
0:00:00.662691
2021-09-13 12:48:34.639414
676879
2021-09-13 12:48:35.223024
605288
2021-09-13 12:48:35.762177
541645
2021-09-13 12:48:36.391932
502202
2021-09-13 12:48:36.972216
605540
2021-09-13 12:48:37.578220
608718
2021-09-13 12:48:38.135008
592885
2021-09-13 12:48:38.808630
669203
2021-09-13 12:48:39.387174
596057
2021-09-13 12:48:39.961275
596129
2021-09-13 12:48:40.605504
642207
2021-09-13 12:48:41.257433
500871
2021-09-13 12:48:41.911872
641778
2021-09-13 12:48:42.463426
642547
2021-09-13 12:48:43.124009
519326


2021-09-13 12:50:25.646849
608336
2021-09-13 12:50:26.309618
656756
2021-09-13 12:50:26.857711
543309
2021-09-13 12:50:27.423930
476595
2021-09-13 12:50:28.019084
661563
2021-09-13 12:50:28.614149
572228
2021-09-13 12:50:29.270235
650633
2021-09-13 12:50:29.852599
641482
2021-09-13 12:50:30.401195
596059
2021-09-13 12:50:30.981348
642180
2021-09-13 12:50:31.624026
593974
away_name       Tampa Bay Rays
home_name    Toronto Blue Jays
Name: 6, dtype: object
2021-09-13 12:50:32.480039
643256
0:00:00.631935
2021-09-13 12:50:33.050575
672386
0:00:00.597306
2021-09-13 12:50:33.683089
666201
0:00:00.623243
2021-09-13 12:50:34.295884
666182
0:00:00.651250
2021-09-13 12:50:34.944620
591971
0:00:00.607283
2021-09-13 12:50:35.542646
572816
0:00:00.613027
2021-09-13 12:50:36.123614
643376
0:00:00.588925
2021-09-13 12:50:36.742620
543807
0:00:00.609051
2021-09-13 12:50:37.410040
547943
0:00:00.647421
2021-09-13 12:50:38.018497
571875
0:00:00.628452
2021-09-13 12:50:38.564457
502481
0:00:00.570919
20

0:00:00.570298
2021-09-13 12:52:23.539007
660896
0:00:00.591811
2021-09-13 12:52:24.112447
593871
0:00:00.613637
2021-09-13 12:52:24.737051
518626
0:00:00.611410
2021-09-13 12:52:25.332831
663558
0:00:00.553492
2021-09-13 12:52:25.894319
573668
0:00:00.575514
2021-09-13 12:52:26.441053
607457
0:00:00.528684
2021-09-13 12:52:26.988005
650333
0:00:00.610160
2021-09-13 12:52:27.614820
608652
0:00:00.555833
2021-09-13 12:52:28.195933
596146
0:00:00.644943
2021-09-13 12:52:28.831903
501381
0:00:00.579491
2021-09-13 12:52:29.700499
593934
0:00:00.943843
2021-09-13 12:52:30.378794
624503
0:00:00.647344
2021-09-13 12:52:31.051024
621248
0:00:00.653667
2021-09-13 12:52:31.690237
608701
0:00:00.642214
2021-09-13 12:52:32.366022
680777
0:00:00.694789
2021-09-13 12:52:33.116960
608648
0:00:00.705961
2021-09-13 12:52:33.735310
553902
0:00:00.653623
2021-09-13 12:52:34.633946
650644
2021-09-13 12:52:35.245009
622065
2021-09-13 12:52:35.814261
642708
2021-09-13 12:52:36.427236
665926
2021-09-13 12:52

2021-09-13 12:54:20.201834
665625
2021-09-13 12:54:20.794598
608686
2021-09-13 12:54:21.349508
642545
2021-09-13 12:54:21.942969
502593
2021-09-13 12:54:22.521659
676083
2021-09-13 12:54:23.075577
665120
2021-09-13 12:54:23.671988
623474
2021-09-13 12:54:24.256165
666176
2021-09-13 12:54:24.906015
650671
2021-09-13 12:54:25.515611
660761
2021-09-13 12:54:26.174653
501571
2021-09-13 12:54:26.782906
448855
2021-09-13 12:54:27.378042
642221
2021-09-13 12:54:27.959660
435559
2021-09-13 12:54:28.549638
667725
2021-09-13 12:54:29.129700
650859
2021-09-13 12:54:29.680222
545358
2021-09-13 12:54:30.485810
594577
2021-09-13 12:54:31.170611
661383
2021-09-13 12:54:31.742163
676050
2021-09-13 12:54:32.294427
594838
2021-09-13 12:54:32.868689
628452
2021-09-13 12:54:33.444847
572125
2021-09-13 12:54:33.993013
660271
2021-09-13 12:54:34.641666
518553
away_name        San Diego Padres
home_name    San Francisco Giants
Name: 13, dtype: object
2021-09-13 12:54:35.442843
543101
0:00:00.626747
2021-09-1

In [477]:
recommendations = run_model_todays_data(batting_stats)

from collections import Counter
Counter(recommendations)

Counter({'Alexander Wells - Baltimore Orioles': 1,
         'Marcus Semien - Toronto Blue Jays': 1,
         'Corey Seager - Los Angeles Dodgers': 9,
         'Skye Bolt - Oakland Athletics': 1,
         'Mookie Betts - Los Angeles Dodgers': 4,
         'Adam Wainwright - St. Louis Cardinals': 2,
         'Tayler Saucedo - Toronto Blue Jays': 1,
         'Cedric Mullins - Baltimore Orioles': 1,
         'Ryan Zimmerman - Washington Nationals': 1,
         'Jose Ramirez - Cleveland Indians': 1,
         'Andrew Heaney - New York Yankees': 1,
         'Tyler Wade - New York Yankees': 1,
         'Miguel Rojas - Miami Marlins': 1,
         'Antonio Senzatela - Colorado Rockies': 1,
         'Gio Urshela - New York Yankees': 1})

In [478]:
datetime.now() - start

datetime.timedelta(seconds=611, microseconds=58055)

## Daily Runtimes

9/11/21 - 564 seconds